Colab 直接打开 

<a href="https://colab.research.google.com/github/sndnyang/LearnIsFun/blob/master/educhat/TryEduchat-qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Educhat Qwen版 

In [ ]:
!pip install transformers==4.41.1 tokenizers==0.19.1 transformers_stream_generator==0.0.5 sentencepiece==0.2.0 accelerate==0.30.1
!pip install tiktoken==0.7.0   einops==0.8.0 
#  !pip install flash-attn --no-build-isolation     # 据说会加速, 但我本地没安装成功又不影响使用，就注释掉了。

In [ ]:
# 海外、colab 不需要， 但国内大概需要
!export HF_ENDPOINT=https://hf-mirror.com

# 路径， 我就默认 colab的路径， 需要相应修改一下。
!huggingface-cli download --resume-download  ecnu-icalk/educhat-sft-002-1.8b-qwen1.5   --local-dir /content/sample_data/educhat-sft-1.8b-qwen1.5
 

In [ ]:
# 补上缺失的文件--可能也需要手动下载，翻墙失败的话
!wget https://huggingface.co/Qwen/Qwen1.5-1.8B-Chat/resolve/main/merges.txt -P   /content/sample_data/educhat-sft-1.8b-qwen1.5/       # 这个是必需的
!wget https://huggingface.co/Qwen/Qwen1.5-1.8B-Chat/resolve/main/tokenizer.json -P /content/sample_data/educhat-sft-1.8b-qwen1.5/        # 这个补了，没看

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
llmpath = '/content/sample_data'

model = AutoModelForCausalLM.from_pretrained(
    os.path.join(llmpath, "educhat-sft-1.8b-qwen1.5"),
    torch_dtype="auto",
    device_map="auto"
)
# GPU 显存使用 8.8G
tokenizer = AutoTokenizer.from_pretrained(os.path.join(llmpath, "educhat-sft-1.8b-qwen1.5"))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
system_prompt = \
"<|system|>"'''你是一个人工智能助手，名字叫EduChat。
- EduChat是一个由华东师范大学开发的对话式语言模型。
EduChat的工具
- Web search: Disable.
- Calculators: Disable.
EduChat的能力
- Inner Thought: Disable.
对话主题
- General: Disable.
- Psychology: Disable.
- Socrates: Enable.'''"</s>"

query = system_prompt + "</s><|prompter|>:给我推荐几本心理相关的书籍</s><|assistant|>:"
inputs = tokenizer(query, return_tensors="pt", padding=True).to(0)
outputs = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.8, repetition_penalty=1.02, max_new_tokens=64)
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(response)

# 从结果来看， 比 13B的百川是差一些
# 更大的 Qwen 没试

好的，我可以向您推荐一些心理相关的书籍。根据您的需求，我可以为您推荐一些适合的心理学或心理学相关的书籍。例如，如果您想要了解心理健康方面的知识，我可以向您推荐《心理学与生活》、《心理治疗师的工作方法》等书籍。如果您想要了解心理学的基本概念和理论


## 通义原模型（可略过）

https://huggingface.co/Qwen/Qwen1.5-1.8B-Chat
